
## NTRU-Encrypt

## Parâmetros
Vamos usar
+ $d \ge 6$ --
  parâmetro de segurança
+ $N$ --
  um primo maior ou igual  2^d
  
+ $p$ --
  um pequeno primo  3,  5 ou  7
 
+ $q$ --
  um primo da ordem de grandeza de  $p*N$
 

 

In [125]:
import hashlib
import random

d = 10

N = next_prime(1 << d)
p = 3
q = next_prime(p*N)

print d, q, p, N

10 3109 3 1031


### anéis de polinómios

In [126]:
Z.<x>  = ZZ[]        # polinómios de coeficientes inteiros
Q.<x>  = PolynomialRing(GF(q),name='x').quotient(x^N-1)

### geração aleatória, arredondamento módulo $q$ e compração módulo $p$

In [127]:
def vec(n):
    return  [choice([-1,0,1]) for k in range(n)]

def seeded(rseed):
    random.seed(rseed)
    return [random.choice([-1,0,1]) for k in range(N)]

# arredondamento módulo 'q'
def qrnd(f):    # argumento em 'Q'
    qq = (q-1)//2 ; ll = map(lift,f.list())
    return [n if n <= qq else n - q  for n in ll]

# arredondamento módulo 'p'
def prnd(l):
    pp = (p-1)//2
    rr = lambda x: x if x <= pp else x - p        
    return [rr(n%p) if n>=0 else -rr((-n)%p) for n in l]

# comparação módulo 'p'
#def equalp(x,y,p):
#    return  all([(a-b)%p == 0  for (a,b) in zip(x,y)] )

def hash_msg(m):
    string_m = ''.join([str(x) for x in m])
    hashed_mess = hashlib.sha512(string_m)
    return hashed_mess

### classe NTRU

In [161]:
class NTRU(object):
    
    #KeyGen
    def __init__(self):
        # calcular um 'f' invertível
        f = Q(0)
        while not f.is_unit():
            F = Q(vec(N)); f = 1 + p*F
        # gerar as chaves
        G = Q(vec(N)) ; g = p*G
        self.f = f
        self.h = f^(-1) * g
    
    def padding(msg):
        print(msg)
        msg = ''.join("{0:08b}".format(x) for x in msg)
        print(msg)
        messlen = len(msg)/8
        print(messlen)
        
        if messlen >= (N-173):
            print("Message is too large")
            return;
        if messlen <= 0:
            print("Message length is invalid")
            return;
            
            
    
        messlen = "{0:06b}".format(int(messlen))
        print(messlen)
        messlen = [int(d) for d in messlen[:6]];
        print(messlen)
        
        msg = list(msg)
        
        m = msg + vec(167) + messlen
    
        return m
        
        
    def encrypt(self,m):
        #pad
        m = padding(m)
        #hash message with pub_key
        rseed = str(hash_msg(m)) + str(hash_msg(self.h))
        #Instantiate Sampler with T and tseed
        r = seeded(rseed)
        r = Q(r)
        t = r * self.h
        tseed=  str(hash_msg(t))
        #Instantiate Sampler with T and tseed
        mask = seeded(tseed)
        mask = Q(mask)
        m = ''.join(str(x) for x in m)
        ml = prnd(qrnd(Q(m) - mask))
        c = t + ml
        
        return c

    def decrypt(self,e):
        ml = prnd(qrnd(self.f + e))
        t = e - Q(ml)
        tseed = str(hash_msg(t))
        #Instantiate sampler with T and tseed
        mask = seeded(tseed)
        mask = Q(mask)
        m = prnd(ml + mask)
        rseed = str(hash_msg(m)) + str(hash_msg(self.h))
        #Instantiate sampler with T and rseed
        r = seeded(rseed)
        r = Q(r)
        #msg,mlen = extract(m) ?
        
        '''
        if p * self.h * r == t:
            result = (msg,mlen)
            return result
        else:
            return -1
        '''
        
        #a = e*self.f
       # return prnd(qrnd(a))


### Teste

In [162]:
m = vec(10)
K = NTRU()
c = K.encrypt(m)
d = K.decrypt(c)

[1, -1, 1, 0, 1, 0, 0, -1, 1, -1]
00000001-00000010000000100000000000000010000000000000000-000000100000001-0000001
10
001010
[0, 0, 1, 0, 1, 0]


/usr/lib/python2.7/dist-packages/sage/misc/sage_eval.py:203: DeprecationWarning: use 0o as octal prefix instead of 0
If you do not want this number to be interpreted as octal, remove the leading zeros.
See http://trac.sagemath.org/17413 for details.
  return eval(source, sage.all.__dict__, locals)


TypeError: unsupported operand type(s) for +: 'PolynomialQuotientRing_generic_with_category.element_class' and 'list'